In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
np.set_printoptions(suppress=True)
import math
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
class MyGaussianNB(BaseEstimator, ClassifierMixin):
    
    def fit(self, X, y):

        self.X = X
        self.y = y

        self.__mean_std_dict = {}
        
        # find each unique class and for each of these classes
        classes = np.unique(y)
        for c in classes:
            # get all indicies for this class
            indices = np.where(y==c)
            
            self.__mean_std_dict[c] = {'prior_prob':(len(indices[0])/len(X))}
            
            # for each column of this class, find the mean and std
            for i in range(len(X[0])):
                
                # for the current class and the current column, find the inidices of the 
                # training set where the class is equal to the current class and 
                # the current column is not null
                notNull = np.where((y==c)&(np.isnan(X[:,i])==False))
                
                # calculate the mean and std based on these values
                self.__mean_std_dict[c][i] = {'mean':(X[notNull,i].mean()), 'std':(X[notNull,i].std())}
            
        return self
        
    def predict(self, X):
        
        self.X = X
        
        predictions = np.empty([0,len(X)])
        
        def conditionalProba(val, mean, stdev):
            '''
            Function that takes a value, a mean value and a standard deviation value.
            It inputs these into a formula supplied in the assignment question and outputs the
            resulting conditional probability for each value.
            '''
            
            if stdev == 0:
                if val == mean:
                    return 1
                else:
                    return 0
                
            probability = (1/(2*np.pi*(stdev**2))**0.5) * np.exp(-((val-mean)**2)/(2*(stdev**2)))
            return probability
        
        classes = self.__mean_std_dict.keys()
        
        # go through each row of the training data
        # for each row
            # make a dictionary for probabilities for each possible class
            
        for i in range(len(X)):
            probDict = {}
            currentRow = X[i]
            
            # for each possible class
                # add a list that includes the current class's prior probability to current
                # class key in the dictionary
            for c in classes:
                probDict[c] = [self.__mean_std_dict[c]['prior_prob']]
                
                # for each column value in the current row
                    # find the mean and std for this column and class in the mean_std_dict
                    # plug the current column value, the mean for this column and the std
                    # for this column into the conditionalProba() function
                    # append the value returned from this to the current class key in the
                    # probability dictionary
                for j in range(len(currentRow)):
                    val = currentRow[j]
                    if math.isnan(val):
                        continue
                    mean = self.__mean_std_dict[c][j]['mean']
                    std = self.__mean_std_dict[c][j]['std']
                    probDict[c].append(conditionalProba(val, mean, std))
                    
                # when the conditional probabilities for each column have been added to the dictionary
                # get the product of each of these values and assign it to the probability dictionary
                # for the current class
                probDict[c] = np.prod(probDict[c])
                
            # normalise the probabilities for each class
            total = sum(probDict.values())
            for k in probDict.keys():
                val = probDict[k]
                probDict[k] = val/total
                
            # get the class name with the highest normalised probability from the dictionary values
            # append the class name to the list of predictions
            index = list(probDict.values()).index(max(probDict.values()))
            predictions = np.append(predictions, list(probDict.keys())[index])
            
        # return the list of predictions
        return predictions

### Changes made to MyGaussianNB() object to allow for missing values:
- In the training phase, when calculating the mean and standard deviation of each feature given the class, just the entries that have a numerical value for the feature are used. That is to say, if the feature value is NaN, then it is not used in caluclating the mean and standard deviation.
- In the prediction phase, when calculating the conditional probability for each feature for each class, once again just the entries that have a numerical value for the feature are used. If the feature has a value of NaN, it is not used in calculating the conditional probability. 

### Process:
- I will be creating models for 2 datasets: PenguinsMV0.2.csv and PenguinsMV0.4.csv, using the MyGaussianNB class defined above.
- As each of the datasets are small, I will evaluate each model's performance using 5-fold cross-validation. This will give a more robust evaluation of each model's performance.
- I will compare the models created by MyGaussianNB to models created with scikit-learn's GaussianNB class across the following scores:
    - accuracy
    - confusion matrices
    - precision
    - recall
    - f1 score
    - balanced accuracy score (as each of the datasets is skewed towards one or more results)

### Read and format penguins data missing 20% of its values

In [ ]:
penguins20 = pd.read_csv("PenguinsMV0.2.csv", index_col = 0)
y = penguins20.pop("species")
X = penguins20

In [ ]:
# substitute "?" for NaN
for col in X.columns:
    X.loc[X[col]=="?", col] = np.NaN

In [ ]:
# convert types to float
X['bill_length'] = X['bill_length'].astype('float64')
X['bill_depth'] = X['bill_depth'].astype('float64')
X['flipper_length'] = X['flipper_length'].astype('float64')
X['body_mass'] = X['body_mass'].astype('float64')

In [ ]:
X = X.to_numpy()

### Imputation versus my own implementation

In [ ]:
myGaussianPipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', MyGaussianNB())])

In [ ]:
UVgaussianNBpipe = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.NaN, strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [ ]:
MVgaussianNBpipe = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=19, random_state=0)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [ ]:
# penguins_20

data = "penguins_20"
pipeLines = [myGaussianPipe, UVgaussianNBpipe, MVgaussianNBpipe]
pipeNames = ["myGaussianPipe", "uniVariateGaussianNBpipe", "multiVariateGaussianNBpipe"]

print(f'Classes are {y.unique()}')

for i in range(len(pipeLines)):
    m = pipeLines[i]
    print("------------------------------------")
    print(pipeNames[i], "data:", data)
    print("------------------------------------")
    accuracy = cross_val_score(m, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    print(f"accuracy: {accuracy.mean()}")
    
    y_pred = cross_val_predict(m, X, y, cv=5)
    conf_mat = confusion_matrix(y, y_pred)
    print(f"confusion matrix: \n{conf_mat}")
    
    precision = precision_score(y, y_pred, average='weighted')
    print(f"precision score: {precision}")
    
    recall = recall_score(y, y_pred, average='weighted')
    print(f"recall score: {recall}")
    
    f1 = f1_score(y, y_pred, average='weighted')
    print(f"f1 score: {f1}")
    
    bas = balanced_accuracy_score(y, y_pred)
    print(f"balanced accuracy score: {bas}")
    print("------------------------------------")
    
    print()

### Imputation versus my own implementation with data missing 20% of its values – conclusions:
- As we can see the MyGaussianNB class performed slightly better across all metrics than the GaussianNB class using both uni-variate and multi-variate imputation.
- As we can see from the confusion matrix, the MyGaussianNB class performed better overall, although the GaussianNB class using multivariate imputation performed slightly better in predicting the second class with one more true positive than the MyGaussianNB class.

### Read and format penguins data missing 40% of its values

In [ ]:
penguins40 = pd.read_csv("PenguinsMV0.4.csv", index_col = 0)

In [ ]:
y = penguins40.pop("species")
X = penguins40

In [ ]:
# substitute "?" for NaN
for col in X.columns:
    X.loc[X[col]=="?", col] = np.NaN

In [ ]:
X['bill_length'] = X['bill_length'].astype('float64')
X['bill_depth'] = X['bill_depth'].astype('float64')
X['flipper_length'] = X['flipper_length'].astype('float64')
X['body_mass'] = X['body_mass'].astype('float64')

In [ ]:
X = X.to_numpy()

### Imputation versus my own implementation

In [ ]:
myGaussianPipe = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('classifier', MyGaussianNB())])

In [ ]:
UVgaussianNBpipe = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.NaN, strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [ ]:
MVgaussianNBpipe = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=33, random_state=0)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])

In [ ]:
# penguins_40

data = "penguins_40"
pipeLines = [myGaussianPipe, UVgaussianNBpipe, MVgaussianNBpipe]
pipeNames = ["myGaussianPipe", "uniVariateGaussianNBpipe", "multiVariateGaussianNBpipe"]

print(f'Classes are {y.unique()}')

for i in range(len(pipeLines)):
    m = pipeLines[i]
    print("------------------------------------")
    print(pipeNames[i], "data:", data)
    print("------------------------------------")
    accuracy = cross_val_score(m, X, y, cv=5, scoring='accuracy', n_jobs=-1)
    print(f"accuracy: {accuracy.mean()}")
    
    y_pred = cross_val_predict(m, X, y, cv=5)
    conf_mat = confusion_matrix(y, y_pred)
    print(f"confusion matrix: \n{conf_mat}")
    
    precision = precision_score(y, y_pred, average='weighted')
    print(f"precision score: {precision}")
    
    recall = recall_score(y, y_pred, average='weighted')
    print(f"recall score: {recall}")
    
    f1 = f1_score(y, y_pred, average='weighted')
    print(f"f1 score: {f1}")
    
    bas = balanced_accuracy_score(y, y_pred)
    print(f"balanced accuracy score: {bas}")
    print("------------------------------------")
    
    print()

### Imputation versus my own implementation with data missing 40% of its values – conclusions:
- We can see here that the three classes performed worse when the data was missing 40% of its values compared with 20%.
- As we can see the MyGaussianNB class performed slightly better across all metrics than the GaussianNB class using both uni-variate and multi-variate imputation.
- As we can see from the confusion matrix, the MyGaussianNB class performed better than the GaussianNB class with multi-variate imputation in predicting the first class with 134 true positives compared to 122, although the GaussianNB class using multivariate imputation performed slightly better in predicting the the second and third classes.
- The MyGaussianNB class performed better than the GaussianNB class with uni-variate imputation in predicting the second and third classes, but slightly worse in predicting the first class.


## Final conclusions:
- The MyGaussianNB class performed notably better than the GaussianNB class using and uni-variate and multi-variate imputation in both multiclass datasets. The performance of both classifiers dropped when tested on a dataset with more missing values, but the MyGaussianNB class still outperformed the GaussianNB class using both imputations in all metrics.
- This leads to my over all conclusion that the MyGaussianNB classifier is a more accurate classifier than the scikit-learn GaussianNB classifier.